In [40]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import ast
from IPython.display import clear_output



In [111]:
# Steg 1: Ladda dataset
training_data = pd.read_csv("training_knn.csv")
exercise_data = pd.read_csv('gym.csv', delimiter=';')
important_data = exercise_data[['Exercise Name', 'Main_muscle', 'Mechanics', 'Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Difficulty (1-5)', 'Secondary Muscles']].copy()
encode_columns = ['Main_muscle', 'Mechanics','Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Secondary Muscles']
ID_columns = ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", "ID_Secondary Muscles"]

important_data.head()

,Exercise Name,Main_muscle,Mechanics,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Difficulty (1-5),Secondary Muscles
0,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
1,Military Press,Shoulder,Compound,"Anterior Deltoid,","Pectoralis Major, Clavicular, Triceps Brachii,...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head, Biceps Brachii, Short Head,",4,"Lateral Deltoid, Triceps Brachii"
2,Military Press: Seated,Shoulder,Compound,"Anterior Deltoid,","Deltoid, Lateral, Supraspinatus, Triceps Brach...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head,",3,"Lateral Deltoid, Triceps Brachii"
3,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
4,Front Raise: Alternating,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"


In [136]:

def encoded_data(specific_muscle):
    predict_data = important_data.copy()
    target_muscle_mapping = []  # List to store muscle names at correct indices
    
    for column in encode_columns:
        predict_data.loc[:, column] = predict_data[column].fillna("")
        predict_data.loc[:, column] = predict_data[column].apply(lambda x: x.split(", ") if x else [])
        
        # Multi-label binarization
        mlb = MultiLabelBinarizer()
        binary_matrix = mlb.fit_transform(predict_data[column])
        
        # Save muscle names in a list if the column is "Target_Muscles"
        if column == "Target_Muscles":
            target_muscle_mapping = list(mlb.classes_)        
        predict_data.loc[:, f"ID_{column}"] = binary_matrix.tolist()
        predict_data = predict_data.drop(columns=[column])
    finished_data = pd.DataFrame()
    for i in range(len(target_muscle_mapping)):
        muscle_index=[]
        if target_muscle_mapping[i].replace(",","") == specific_muscle:
            muscle_index.append(i)
        for index in muscle_index:
            temporary_dataset = predict_data[predict_data["ID_Target_Muscles"].apply(lambda x: x[index] == 1)]
            finished_data = pd.concat([finished_data, temporary_dataset], ignore_index=True)
    return finished_data


In [100]:
def knn(predict_data,user_fitness_level):
    for col in ID_columns:
        training_data[col] = training_data[col].apply(ast.literal_eval).apply(np.array)

    flattened_features = []
    for col in ID_columns:
        arrays = np.stack(training_data[col].values)  
        array_df = pd.DataFrame(arrays, columns=[f"{col}_{i}" for i in range(arrays.shape[1])])
        flattened_features.append(array_df)

    data_flattened = pd.concat([training_data.drop(ID_columns, axis=1)] + flattened_features, axis=1)

    X = data_flattened.drop(["Exercise Name", "Difficulty (1-5)", "score"], axis=1)
    y = data_flattened["score"]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_scaled, y)

    for col in ID_columns:
        predict_data[col] = predict_data[col].apply(ast.literal_eval).apply(np.array)

    flattened_features_test = []
    for col in ID_columns:
        arrays_test = np.stack(predict_data[col].values)
        array_df_test = pd.DataFrame(arrays_test, columns=[f"{col}_{i}" for i in range(arrays_test.shape[1])])
        flattened_features_test.append(array_df_test)

    data_test_flattened = pd.concat([predict_data.drop(ID_columns, axis=1)] + flattened_features_test, axis=1)

    X_test = data_test_flattened.drop(["Exercise Name", "Difficulty (1-5)"], axis=1)
    X_test_scaled = scaler.transform(X_test)

    y_pred_test = knn.predict(X_test_scaled)

    def adjust_score(predicted_scores, user_fitness_level, exercise_difficulty, max_difference=4):
        adjusted_scores = []
        for pred, difficulty in zip(predicted_scores, exercise_difficulty):
            factor = 1 - (abs(user_fitness_level - difficulty) / max_difference)**2
            factor = max(0.5,factor) 
            adjusted_scores.append(pred * factor) 
        return adjusted_scores

    exercise_difficulty = data_test_flattened["Difficulty (1-5)"]
    adjusted_scores_test = adjust_score(y_pred_test, user_fitness_level, exercise_difficulty)

    data_test_flattened["Predicted Score"] = adjusted_scores_test

    sorted_results = data_test_flattened.sort_values(by="Predicted Score", ascending=False)
    return sorted_results["Exercise Name"].iloc[0]
 



In [ ]:


# Display the list of main muscles with numbers
def display_muscles(muscles):
    print("Select one or more main muscles from the list:")
    for index, muscle in enumerate(muscles, start=1):
        print(f"{index}. {muscle}")

# Select main muscles and filter target muscles
def select_MainMuscles(data):
    available_muscles = data['Main_muscle'].unique().tolist()  # List of available main muscles
    selected_muscles = set()  # Set to keep track of selected muscles for the current day

    selecting = True
    while selecting:
        # Display available main muscles
        print("Select one or more main muscles from the list:")
        for index, muscle in enumerate(available_muscles, start=1):
            print(f"{index}. {muscle}")

        choice = input("Enter the number of the muscle you want to select (or 'q' to finish selection): ")

        if choice.lower() == 'q':
            selecting = False
        
        else:
            try:
                choice_index = int(choice) - 1
                if 0 <= choice_index < len(available_muscles):
                    selected_muscle = available_muscles[choice_index]
                    if selected_muscle in selected_muscles:
                        print(f"You have already selected: {selected_muscle}. Please choose another muscle.")
                    else:
                        print(f"You have selected: {selected_muscle}")
                        selected_muscles.add(selected_muscle)  # Add to the set to prevent future selections for today
                else:
                    print("Invalid choice, please try again.")
            except ValueError:
                print("Please enter a valid number.")

    return selected_muscles  # Return the set of selected main muscles for the day

# Filter target muscles based on selected main muscles
def filter_TargetMuscles(data, selected_muscles):
    target_muscles = []  # List to store target muscles
    for muscle in selected_muscles:
        filtered_data = data[data['Main_muscle'] == muscle]
        for _, target in filtered_data.iterrows():
            target_name = target['Target_Muscles']  # Corrected column name
            if target_name not in target_muscles:  # Check for duplicates
                target_muscles.append(target_name)  # Append target muscles to the list

    return target_muscles  # Return the list of target muscles

# Function to handle pre-planned workouts
def PREplanned_workout():
    print("Select a pre-planned workout type:")
    print("1. Push")
    print("2. Pull")
    print("3. Leg")
    print("4. Whole Body")

    while True:
        choice = input("Enter the number of the workout type you want to select: ")

        try:
            choice_index = int(choice)
            if choice_index == 1:  # Push
                main_muscles = ['Chest', 'Shoulder', 'Triceps']
                print("You have selected: Push")
                return main_muscles
            elif choice_index == 2:  # Pull
                main_muscles = ['Back', 'Biceps', 'Forearm']
                print("You have selected: Pull")
                return main_muscles
            elif choice_index == 3:  # Leg
                main_muscles = ['Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Leg")
                return main_muscles
            elif choice_index == 4:  # Whole Body
                main_muscles = ['Chest', 'Back', 'Shoulder', 'Biceps', 'Triceps', 'Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Whole Body")
                return main_muscles
            else:
                print("Invalid choice, please try again.")
        except ValueError:
            print("Please enter a valid number.")

# Function to select difficulty level
def select_difficulty():
    print("Select difficulty level")
    while True:
        difficulty = input("Select difficulty level (1-5): ")
        try:
            difficulty = int(difficulty)
            if 1 <= difficulty <= 5:
                print(f"You have selected difficulty level: {difficulty}")
                return difficulty  # Return the selected difficulty level
            else:
                print("Please enter a number between 1 and 5.")
        except ValueError:
            print("Please enter a valid number.")


file_path = 'gym.csv'  # Path to your CSV file
# Load the CSV file
data = pd.read_csv(file_path,delimiter=';')

# Dictionary to store workouts for each day of the week
weekly_workouts = {}

# List of days in a week
days_of_week = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

# Ask for difficulty level once for the whole week
difficulty = select_difficulty()  # Get the difficulty level for the week

for day in days_of_week:
    clear_output()
    print(f"{day}:")
    print("Select workout type:")
    print("1. Pre-planned")
    print("2. Hybrid")
    print("3. Rest Day")  # Added Rest Day option here

    while True:  # Loop until a valid choice is made
        workout_choice = input("Enter the number of the workout type you want to select: ")
        
        try:
            workout_choice_index = int(workout_choice)  # Try to convert input to an integer
            if workout_choice_index == 1:  # Pre-planned
                main_muscles = PREplanned_workout()
                break  # Exit the inner loop if a valid choice is made
            elif workout_choice_index == 2:  # Hybrid
                main_muscles = select_MainMuscles(data)
                break  # Exit the inner loop if a valid choice is made
            elif workout_choice_index == 3:  # Rest Day
                print("Rest day selected. No workout planned.")
                weekly_workouts[day] = []  # Set to an empty list for Rest Day
                break  # Exit the inner loop
            else:
                print("Invalid choice, please try again.")  # Prompt for valid input again
        except ValueError:
            print("Please enter a valid number.")  # Handle non-integer inputs
   
    # Collect all unique target muscles for the selected main muscles
    if workout_choice_index != 3:  # Only filter target muscles if it's not a Rest Day
        selected_targets = filter_TargetMuscles(data, main_muscles)

        # Store the selected target muscles in the dictionary
        if selected_targets:
            weekly_workouts[day] = selected_targets  # Store as a dictionary
clear_output()





ParserError: Error tokenizing data. C error: Expected 18 fields in line 3, saw 23


In [45]:
# Print the weekly workouts
print("Weekly Workouts:")
print(weekly_workouts)
print(f"Difficulty level for the week is set to: {difficulty}")



Weekly Workouts:
{'MONDAY': ['Pectoralis Major Sternal,', 'Pectoralis Major Sternal', 'Pectoralis Major Clavicular,', 'Serratus Anterior,', 'Anterior Deltoid,', 'Lateral Deltoid,', 'Posterior Deltoid,', 'Supraspinatus,', 'Triceps Brachii,'], 'TUESDAY': [], 'WEDNESDAY': [], 'THURSDAY': [], 'FRIDAY': [], 'SATURDAY': [], 'SUNDAY': []}
Difficulty level for the week is set to: 1


In [142]:

test = encoded_data(weekly_workouts["MONDAY"][0].replace(",", ""))
knn(test,difficulty)


ValueError: malformed node or string: array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [139]:
test = ""
for i in weekly_workouts:
    for k in weekly_workouts[i]:
        k = k.replace(",", "")
        predic_data = encoded_data(k)
        excercise = knn(predic_data,difficulty)
        #print(k)
        #print(excercise)


ValueError: malformed node or string: array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])